In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
trans_df = pd.read_csv('trans_df.csv')

In [2]:
trans_df.drop(columns = 'Unnamed: 0', inplace = True)
trans_df = trans_df[trans_df.ru.notna()]
trans_df.isnull().sum()

ru       0
tyv      0
split    0
dtype: int64

In [3]:
trans_df.split.value_counts()

split
train    108183
test       5000
dev        4999
Name: count, dtype: int64

In [4]:
df_train = trans_df[trans_df.split=='train'].copy()
df_dev = trans_df[trans_df.split=='dev'].copy()
df_test = trans_df[trans_df.split=='test'].copy()

In [5]:
df_train

,ru,tyv,split
0,листья савойской капусты бланшировать в течени...,савой капустазының бүрүлерин 5 минуттуң дургуз...,train
1,Джубаленд,Джубаленд,train
2,доход,орулга,train
3,И теперь можете начать учиться помогать другим...,Ынчангаш өскелерге дузалап база оларның чаагай...,train
4,"После сих дней зачала Елисавета, жена его, и т...","Элээн үе эрткен соонда, ооң кадайы Елисавета и...",train
...,...,...,...
118175,конституция,конституция,train
118178,физический,күш,train
118179,храбрый баран,дидим кошкар,train
118180,Вы вправе верить или не верить учению Библии.,"Библияның өөредиинге бүзүрээрин, бүзүревезин б...",train


In [6]:
df_train_ru = df_train['ru']
df_train_tyv = df_train['tyv']

df_test_ru = df_test['ru']
df_test_tyv = df_test['tyv']

df_dev_ru = df_dev['ru']
df_dev_tyv = df_dev['tyv']

In [7]:
df_train_ru.to_csv('df_train_ru',header='ru', index=False)
df_train_tyv.to_csv('df_train_tyv',header='tyv', index=False)

df_test_ru.to_csv('df_test_ru',header='ru', index=False)
df_test_tyv.to_csv('df_test_tyv',header='tyv', index=False)

df_dev_ru.to_csv('df_dev_ru',header='ru', index=False)
df_dev_tyv.to_csv('df_dev_tyv',header='tyv', index=False)

In [8]:
! git clone https://github.com/awslabs/sockeye.git
! cd sockeye && pip3 install --editable .

Cloning into 'sockeye'...
remote: Enumerating objects: 10184, done.
remote: Counting objects: 100% (352/352), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 10184 (delta 233), reused 268 (delta 176), pack-reused 9832
Receiving objects: 100% (10184/10184), 10.17 MiB | 21.17 MiB/s, done.
Resolving deltas: 100% (7798/7798), done.
Obtaining file:///content/sockeye
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu12

In [9]:
! pip3 install subword-nmt

In [10]:
! cat df_train_ru df_train_tyv |subword-nmt learn-bpe -s 40000 >codes

100% 40000/40000 [01:45<00:00, 379.28it/s]


In [11]:
! subword-nmt apply-bpe -c codes < df_train_ru > df_train_ru_bpe
! subword-nmt apply-bpe -c codes < df_train_tyv > df_train_tyv_bpe

! subword-nmt apply-bpe -c codes < df_test_ru > df_test_ru_bpe
! subword-nmt apply-bpe -c codes < df_test_tyv > df_test_tyv_bpe

! subword-nmt apply-bpe -c codes < df_dev_ru > df_dev_ru_bpe
! subword-nmt apply-bpe -c codes < df_dev_tyv > df_dev_tyv_bpe

In [12]:
df_test_ru_bpe = pd.read_csv('df_test_ru_bpe', on_bad_lines='skip')
df_test_tyv_bpe = pd.read_csv('df_test_tyv_bpe', on_bad_lines='skip')

df_train_ru_bpe = pd.read_csv('df_train_ru_bpe', on_bad_lines='skip')
df_train_tyv_bpe = pd.read_csv('df_train_tyv_bpe', on_bad_lines='skip')

df_dev_ru_bpe = pd.read_csv('df_dev_ru_bpe', on_bad_lines='skip')
df_dev_tyv_bpe = pd.read_csv('df_dev_tyv_bpe', on_bad_lines='skip')

In [14]:
! sockeye-prepare-data \
    --source df_train_tyv_bpe --target df_train_ru_bpe --shared-vocab \
    --word-min-count 1 \
    --output prepared

[INFO:sockeye.utils] Sockeye: 3.1.37, commit unknown, path /content/sockeye/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/usr/local/lib/python3.10/dist-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /usr/local/bin/sockeye-prepare-data --source df_train_tyv_bpe --target df_train_ru_bpe --shared-vocab --word-min-count 1 --output prepared
[INFO:sockeye.utils] Arguments: Namespace(config=None, source='df_train_tyv_bpe', source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_factors_use_target_vocab=[], target='df_train_ru_bpe', end_of_prepending_tag=None, source_vocab=None, target_vocab=None, source_factor_vocabs=[], target_factor_vocabs=[], shared_vocab=True, num_words=(0, 0), word_min_count=(1, 1), pad_vocab_to_multiple_of=8, no_bucketing=False, bucket_width=8, bucket_scaling=False, max_seq_len=(95, 95), num_samples_per_shard=10000000, min_num_shards=1, seed=13, output='prepared', quiet=False, quiet_secondary_workers=False, no_logfi

In [16]:
! torchrun --no_python --nproc_per_node 1 sockeye-train \
    --prepared-data prepared \
    --validation-source df_dev_tyv_bpe --validation-target df_dev_ru_bpe \
    --output model \
    --encoder transformer --decoder transformer --num-layers 6 --num-embed 512 \
    --transformer-model-size 512 --transformer-attention-heads 16 --transformer-feed-forward-num-hidden 2048 \
    --batch-type max-word \
    --batch-size 5000 --update-interval 20 --checkpoint-interval 500 \
    --max-num-epochs 140 --dist \
    --initial-learning-rate 0.1 \
    --optimizer adam --optimized-metric bleu \
    --learning-rate-scheduler-type inv-sqrt-decay --learning-rate-warmup 4000 \
    --seed 1

[INFO:sockeye.utils] Sockeye: 3.1.37, commit unknown, path /content/sockeye/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/usr/local/lib/python3.10/dist-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /usr/local/bin/sockeye-train --prepared-data prepared --validation-source df_dev_tyv_bpe --validation-target df_dev_ru_bpe --output model --encoder transformer --decoder transformer --num-layers 6 --num-embed 512 --transformer-model-size 512 --transformer-attention-heads 16 --transformer-feed-forward-num-hidden 2048 --batch-type max-word --batch-size 5000 --update-interval 20 --checkpoint-interval 500 --max-num-epochs 140 --dist --initial-learning-rate 0.1 --optimizer adam --optimized-metric bleu --learning-rate-scheduler-type inv-sqrt-decay --learning-rate-warmup 4000 --seed 1
[INFO:sockeye.utils] Arguments: Namespace(config=None, source=None, source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_factors_use_target_vocab=[], target=N

In [17]:
! sockeye-translate \
    --input df_test_tyv_bpe \
    --output out.bpe \
    --model model \
    --dtype float16 \
    --beam-size 5 \
    --batch-size 64

[INFO:sockeye.utils] Sockeye: 3.1.37, commit unknown, path /content/sockeye/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/usr/local/lib/python3.10/dist-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /usr/local/bin/sockeye-translate --input df_test_tyv_bpe --output out.bpe --model model --dtype float16 --beam-size 5 --batch-size 64
[INFO:sockeye.utils] Arguments: Namespace(config=None, input='df_test_tyv_bpe', input_factors=None, json_input=False, output='out.bpe', models=['model'], checkpoints=None, nbest_size=1, beam_size=5, greedy=False, beam_search_stop='all', batch_size=64, chunk_size=None, sample=None, seed=None, ensemble_mode='linear', bucket_width=10, max_input_length=None, max_output_length_num_stds=2, max_output_length=None, restrict_lexicon=None, restrict_lexicon_topk=None, skip_nvs=False, nvs_thresh=0.5, strip_unknown_words=False, prevent_unk=False, output_type='translation', length_penalty_alpha=1.0, length_penalty_beta=0.0, brevity_penalty_typ

In [19]:
! sed -re 's/(@@ |@@$)//g' <out.bpe >out.tok
! sacrebleu df_test_ru -tok none -i out.tok

{
 "name": "BLEU",
 "score": 9.4,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:exp|version:2.4.2",
 "verbose_score": "28.0/12.8/6.7/3.7 (BP = 0.970 ratio = 0.970 hyp_len = 19052 ref_len = 19642)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "exp",
 "version": "2.4.2"
}


In [20]:
with open('out.tok', 'r') as file:
    content = file.read()

In [21]:
sentences = content.split('\n')

In [22]:
sentences_df = pd.DataFrame(sentences)
sentences_df

,0
0,ru
1,погребительный обряд
2,Гельма
3,душа
4,волосы катится
...,...
4997,комплимент
4998,ма
4999,обжорливый человек
5000,выпустить в свет


In [23]:
sentences_df.to_csv('sentences_df.csv')

In [24]:
df_test_tyv.to_csv('df_test_tyv.csv')
df_test_ru.to_csv('df_test_ru.csv')

In [25]:
df_test_tyv

25        орнукшудулга ёзулалы
131                     Гельма
143                   чүректиң
197         баш дүгү дүжүп тур
202                хере тыртар
                  ...         
118076            ис чок чидер
118112              комплимент
118135                      ма
118145             хоптак кижи
118176                 үндүрер
Name: tyv, Length: 5000, dtype: object

In [26]:
df_test_ru

25         погребальный отряд
131                    Гельма
143                 сердечный
197              волосы лезут
202                  отогнуть
                 ...         
118076    исчезнуть без следа
118112             комплимент
118135                     на
118145                 обжора
118176                тратить
Name: ru, Length: 5000, dtype: object

not enough values to unpack (expected 3, got 2)
